<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/20250612%20Trust-FrdAvg%20MLP-LSTM_75%20Rounds%20on%20WSN-DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

20250612 Trust FL MLP-LSTM_logged_extended on WSN-DS

In [1]:
# -------------------------------------------------------------
#   TrustFed-IDS  (variant A)  –  WSN-DS  five-class experiment
# -------------------------------------------------------------
import os, time, psutil, numpy as np, pandas as pd, tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import InputLayer, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from scipy.spatial.distance import cosine
from typing import List, Tuple

In [2]:
# Load dataset
dataset = pd.read_csv("dataset_WSN-DS.csv")  # WSN-DS 5-class dataset
dataset = dataset.dropna()  # Remove missing values

In [3]:
dataset

,id,Time,Is_CH,who CH,Dist_To_CH,ADV_S,ADV_R,JOIN_S,JOIN_R,SCH_S,SCH_R,Rank,DATA_S,DATA_R,Data_Sent_To_BS,dist_CH_To_BS,send_code,Expaned Energy,Class
0,101000,50,1,101000,0.00000,1,0,0,25,1,0,0,0,1200,48,130.08535,0,2.46940,0
1,101001,50,0,101044,75.32345,0,4,1,0,0,1,2,38,0,0,0.00000,4,0.06957,0
2,101002,50,0,101010,46.95453,0,4,1,0,0,1,19,41,0,0,0.00000,3,0.06898,0
3,101003,50,0,101044,64.85231,0,4,1,0,0,1,16,38,0,0,0.00000,4,0.06673,0
4,101004,50,0,101010,4.83341,0,4,1,0,0,1,25,41,0,0,0.00000,3,0.06534,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374656,201096,1003,0,201051,6.98337,0,5,1,0,0,1,7,96,0,67,170.14779,3,0.15974,0
374657,201097,1003,0,201037,29.32867,0,5,1,0,0,1,31,39,0,24,82.21043,2,0.06877,0
374658,201098,1003,0,201095,18.51963,0,5,1,0,0,1,17,55,0,31,139.26438,1,0.09437,0
374659,201099,1003,0,201051,8.55001,0,5,1,0,0,1,3,96,0,65,158.27492,3,0.16047,0


In [4]:
# Encode non-numeric columns (if any)
for column in dataset.columns:
    if dataset[column].dtype == 'object':  # Identify categorical columns
        dataset[column] = LabelEncoder().fit_transform(dataset[column])

# Separate features (X) and target (y)
X = dataset.drop(['Class'], axis=1)  # Assuming 'Class' is the target column
y = dataset['Class']

print("Original Class Distribution:", np.bincount(y))

Original Class Distribution: [340066  14596  10049   6638   3312]


In [5]:
from sklearn.preprocessing import LabelEncoder, StandardScaler # Import StandardScaler here
# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

from imblearn.over_sampling import SMOTE

# Apply SMOTE to balance the data
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

# Verify the balanced dataset distribution
print("Balanced Class Distribution:", np.bincount(y))

Balanced Class Distribution: [340066 340066 340066 340066 340066]


In [6]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101, stratify=y)

In [7]:
pip install matplotlib

In [8]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, TimeDistributed
from tensorflow.keras.regularizers import l2
import numpy as np

In [9]:
# ------------------------------- CONFIG --------------------------------
SEED            = 42
NUM_CLIENTS     = 5
ROUNDS          = 75                # more rounds, shallower locals
LOCAL_EPOCHS    = 1
BATCH_SIZE      = 32
DIRICHLET_ALPHA = 0.5
HISTORY_KEEP    = 6
TRUST_ALPHA     = (0.30, 0.55, 0.15)   # sim, loss, stability
LOG_DIR         = "/mnt/data"
DATA_PATH       = "dataset.csv"        # path to WSN-BFSF CSV
# -----------------------------------------------------------------------

# --------------- 0. Reproducibility -----------------------------------
np.random.seed(SEED)
tf.random.set_seed(SEED)

# ------------------ 1. LOAD & PRE-PROCESS DATA -------------------------
df = pd.read_csv("dataset_WSN-DS.csv").dropna()

# encode any object columns
for col in df.select_dtypes(include="object"):
    df[col] = LabelEncoder().fit_transform(df[col])

X_all = df.drop("Class", axis=1).values.astype("float32")
y_all = df["Class"].values.astype("int64")

In [10]:
# ------------------ 1a. TRAIN / TEST SPLIT -----------------------------
X_tr, X_te, y_tr, y_te = train_test_split(
    X_all, y_all, test_size=0.20, stratify=y_all, random_state=SEED)

# ------------------ 1b. Fit scaler + SMOTE on TRAIN only ---------------
scaler = StandardScaler().fit(X_tr)
X_tr   = scaler.transform(X_tr)
X_te   = scaler.transform(X_te)

smote  = SMOTE(random_state=SEED)
X_tr, y_tr = smote.fit_resample(X_tr, y_tr)

# 3-D reshape for LSTM
X_tr = X_tr[..., None]
X_te = X_te[..., None]

num_classes = int(y_tr.max() + 1)
y_te_cat    = to_categorical(y_te, num_classes)

# ---------------- 2. NON-IID DIRICHLET SPLIT ---------------------------
def dirichlet_split(X, y, k, alpha, rng):
    idx_class = {c: np.where(y == c)[0] for c in np.unique(y)}
    client_idx = [[] for _ in range(k)]
    for c, idx in idx_class.items():
        rng.shuffle(idx)
        parts = (rng.dirichlet([alpha]*k) * len(idx)).astype(int)
        while parts.sum() < len(idx):
            parts[rng.randint(0, k)] += 1
        s = 0
        for cid, ct in enumerate(parts):
            client_idx[cid].extend(idx[s:s+ct]); s += ct
    for lst in client_idx: rng.shuffle(lst)
    return [X[l] for l in client_idx], [y[l] for l in client_idx]

rng = np.random.RandomState(SEED)
client_X_raw, client_y_raw = dirichlet_split(X_tr, y_tr, NUM_CLIENTS,
                                             DIRICHLET_ALPHA, rng)
client_X = client_X_raw
client_y = [to_categorical(y, num_classes) for y in client_y_raw]

input_shape = (X_tr.shape[1], 1)

# ---------------- 3. MODEL BUILDER -------------------------------------
# from tensorflow.keras.experimental_api.keras.optimizers.schedules import CosineDecay # Import CosineDecay - REMOVE THIS LINE
from tensorflow.keras.optimizers.schedules import CosineDecay # Import CosineDecay from the correct path
from sklearn.utils.class_weight import compute_class_weight # Import compute_class_weight


def build_model(inp=input_shape, classes=num_classes):
    lr_sched = CosineDecay(initial_learning_rate=5e-4,
                           decay_steps=ROUNDS,
                           alpha=0.4)  # floor = 0.4 × initial = 2e-4
    opt = tf.keras.optimizers.Nadam(learning_rate=lr_sched, clipnorm=2.0)

    m = Sequential([
        InputLayer(input_shape=inp),
        LSTM(128, activation='tanh', return_sequences=True,
             kernel_regularizer=l2(5e-4)),
        LSTM(64,  activation='tanh', kernel_regularizer=l2(5e-4)),
        Dense(256, activation='relu'),
        Dropout(0.20),                       # milder dropout
        Dense(128, activation='relu'),
        Dropout(0.25),
        Dense(classes, activation='softmax')
    ])
    m.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return m

# ---------------- 4. TRUST-FEDAVG UTILITIES ----------------------------
def weight_update(local, global_):
    return [l - g for l, g in zip(local, global_)]

def vec_cos(a, b):
    v1, v2 = np.concatenate([w.ravel() for w in a]), np.concatenate([w.ravel() for w in b])
    return 0.0 if (np.all(v1 == 0) or np.all(v2 == 0)) else 1 - cosine(v1, v2)

def stability(upd, hist):
    if len(hist) < 2: return 1.0
    return float(np.nanmean([vec_cos(upd, h) for h in hist[-HISTORY_KEEP:]]))

def compute_trust(upd, vloss, hist):
    lo, hi = min(vloss.values()), max(vloss.values())
    trust = {}
    for cid, u in upd.items():
        score = (TRUST_ALPHA[0]*vec_cos(u, [np.zeros_like(w) for w in u]) +
                 TRUST_ALPHA[1]*(1 - (vloss[cid]-lo)/(hi-lo+1e-8)) +
                 TRUST_ALPHA[2]*stability(u, hist[cid]))
        trust[cid] = max(score, 1e-6)
    return trust

def aggregate(w, t, n):
    tot = sum(t[c]*n[c] for c in w)
    return [sum(t[c]*n[c]*w[c][l] for c in w)/tot
            for l in range(len(next(iter(w.values()))))]

# ---------------- 5. INITIALISE ----------------------------------------
g_model   = build_model()
g_weights = g_model.get_weights()
model_MB  = sum(w.nbytes for w in g_weights)/2**20

history   = {c: [] for c in range(NUM_CLIENTS)}
perf_log, comm_log, trust_log = [], [], []

# class-weight dictionary (balanced)
cls_wt = compute_class_weight('balanced',
                              classes=np.arange(num_classes), y=y_tr)
class_weight = dict(enumerate(cls_wt))

# ---------------- 6. FEDERATED TRAINING --------------------------------
for r in range(1, ROUNDS+1):
    tic = time.time()
    lw, upd, vloss, ns, bytes_out = {}, {}, {}, {}, 0

    for cid in range(NUM_CLIENTS):
        n_val = max(1, int(0.1*len(client_X[cid])))
        Xv, yv = client_X[cid][:n_val], client_y[cid][:n_val]
        Xt, yt = client_X[cid][n_val:], client_y[cid][n_val:]

        local = build_model(); local.set_weights(g_weights)
        local.fit(Xt, yt, epochs=LOCAL_EPOCHS, batch_size=BATCH_SIZE,
                  verbose=0, class_weight=class_weight)

        w = local.get_weights()
        u = weight_update(w, g_weights)
        l = local.evaluate(Xv, yv, verbose=0)[0]

        lw[cid], upd[cid], vloss[cid], ns[cid] = w, u, l, len(Xt)
        history[cid] = (history[cid] + [u])[-HISTORY_KEEP:]
        bytes_out += sum(x.nbytes for x in w)

    trust = compute_trust(upd, vloss, history)
    g_weights = aggregate(lw, trust, ns)
    g_model.set_weights(g_weights)

    # ---- evaluation on test set ----
    y_pred = np.argmax(g_model.predict(X_te, verbose=0), axis=1)
    perf_log.append({
        "round": r,
        "accuracy":  accuracy_score(y_te, y_pred),
        "precision": precision_score(y_te, y_pred, average='weighted', zero_division=0),
        "recall":    recall_score(y_te, y_pred, average='weighted', zero_division=0),
        "f1":        f1_score(y_te, y_pred, average='weighted', zero_division=0),
        "ms": round((time.time()-tic)*1000, 2)
    })
    comm_log.append({"round": r, "MB": bytes_out/2**20})
    trust_log.extend([{"round": r, "client": c, "trust": t}
                      for c, t in trust.items()])

    print(f"R{r:02d}  acc={perf_log[-1]['accuracy']:.8f}  "
          f"F1={perf_log[-1]['f1']:.8f}  MB={bytes_out/2**20:.8f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


R01  acc=0.71890889  F1=0.79136492  MB=3.16904068


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


R02  acc=0.96698384  F1=0.96776156  MB=3.16904068


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


...

R73  acc=0.97792695  F1=0.97965434  MB=3.16904068


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


R74  acc=0.97872766  F1=0.98046188  MB=3.16904068


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


R75  acc=0.97740648  F1=0.97923445  MB=3.16904068


In [11]:
# ---------------- 7. SAVE LOGS -----------------------------------------
os.makedirs(LOG_DIR, exist_ok=True)
pd.DataFrame(perf_log ).to_csv(f"{LOG_DIR}/perf_log_WSN-DS.csv", index=False)
pd.DataFrame(comm_log ).to_csv(f"{LOG_DIR}/comm_log_WSN-DS.csv", index=False)
pd.DataFrame(trust_log).to_csv(f"{LOG_DIR}/trust_log_WSN-DS.csv", index=False)

profile = {
    "Params_MB": round(model_MB,3),
    "Rounds": ROUNDS,
    "Clients": NUM_CLIENTS,
    "PeakMem_MB": round(psutil.Process(os.getpid()).memory_info().rss/2**20,2)
}
pd.DataFrame([profile]).to_csv(f"{LOG_DIR}/model_profile_WSN-DS.csv", index=False)
print("\n✓ Tuned LSTM run complete – logs saved to", LOG_DIR)


✓ Tuned LSTM run complete – logs saved to /mnt/data


In [14]:
# prompt: Download all logs files aboved

from google.colab import files
files.download(f"{LOG_DIR}/perf_log_WSN-DS.csv")
files.download(f"{LOG_DIR}/comm_log_WSN-DS.csv")
files.download(f"{LOG_DIR}/trust_log_WSN-DS.csv")
files.download(f"{LOG_DIR}/model_profile_WSN-DS.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>